In [1]:
import sys
sys.path.append('../src')

In [2]:
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

## Test vision recorder

In [3]:
import gta.vision

In [4]:
recorder = gta.vision.VisionRecorder()
recorder.start()
time.sleep(2)
recorder.stop()

In [5]:
recorder.results.shape

(26, 797, 1366, 3)

In [6]:
print('Mean time between images is %g sec.' % np.mean(np.diff(recorder.times)))

Mean time between images is 0.0705684 sec.


In [ ]:
fig, ax = plt.subplots()
i = -1
ax.imshow(recorder.results[i])
times = recorder.times
ax.set_title('$t=%g$ sec' % (times[i] - times[0],));

## Test keyboard recorder

In [8]:
import gta.keyboard
reload(gta.keyboard);

In [9]:
recorder = gta.keyboard.KeyboardRecorder()
recorder.start()
print('Use the keyboard now ...', end=' ')
time.sleep(10)
recorder.stop()
print('done.')

Use the keyboard now ... done.


abcd 1234

In [10]:
len(recorder.resultsList)

21

In [11]:
t0 = recorder.times[0]
[
    print('%.3f %s %d %d' % (t-t0, recorder.scanCodes[sc], iskp, down))
    for t, (sc, iskp, down) in zip(recorder.times, recorder.results)
];

0.000 enter 0 1
0.132 enter 0 0
0.380 left ctrl 0 1
0.472 a 0 1
0.556 left ctrl 0 0
0.596 a 0 0
0.832 a 0 1
0.984 a 0 0
1.024 b 0 1
1.088 b 0 0
1.204 c 0 1
1.316 c 0 0
1.324 d 0 1
1.436 d 0 0
1.456 space 0 1
1.596 space 0 0
1.602 1 0 1
1.684 2 0 1
1.724 1 0 0
1.792 3 0 1
1.872 2 0 0


## Test gamepad recorder

In [12]:
import gta.gamepad
reload(gta.gamepad)

<module 'gta.gamepad' from '../src\\gta\\gamepad.py'>

In [13]:
recorder = gta.gamepad.GamepadRecorder()

In [14]:
recorder.start()

In [15]:
recorder.worker.is_alive()

True

In [16]:
allKeys = set()
nbuttons = 20
print('Use all %d controls:' % nbuttons)
while True:
    time.sleep(1)
    l = set([n for (t, (n, v)) in recorder.resultsList])
    newKeys = l.difference(allKeys)
    done = False
    if len(newKeys) > 0:
        for eid in newKeys:
            print(eid, end='=')
            print(gta.gamepad.eid2name(eid), end='')
            allKeys.add(eid)
            if len(allKeys) >= nbuttons:
                print('.')
                done = True
            else:
                print(', ')
    if done:
        break

Use all 20 controls:
0=d_up, 
1=d_down, 
2=d_left, 
3=d_right, 
4=start, 
5=back, 
6=l_stick, 
14=l_thumb_x, 
7=r_stick, 
8=l_bumper, 
9=r_bumper, 
10=a, 
11=b, 
12=x, 
13=y, 
15=l_thumb_y, 
16=r_thumb_x, 
17=r_thumb_y, 
18=left_trigger, 
19=right_trigger.


In [17]:
len(recorder.times)

837

In [ ]:
t0 = recorder.times[0]
for (t, (eid, v)) in recorder.resultsList:
    print('%g: %s %s' % (t-t0, gta.gamepad.eid2name(eid), v))

In [19]:
recorder.stop()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
t0 = min(recorder.times)
for name in gta.gamepad.axisNames:
    tv = np.array([
        (t-t0, v)
        for (t, (k, v)) in zip(recorder.times, recorder.results)
         if gta.gamepad.eid2name(k) == name
    ])
    t = tv[:, 0]; v = tv[:, 1]
    ax.plot(t, v, label=name, marker='o', markersize=2)
ax.set_xlabel('$t-t_0$ [sec]')
ax.set_ylabel('value')
ax.legend()